<h3> Data Cleaning </h3>

In [281]:
import numpy as np
import pandas as pd
import fuzzywuzzy
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

In [282]:
delegates = pd.read_excel("Data/constitutional_convention_1787.xlsx", header = 2)
state_delegates = pd.read_excel("Data/State Delegates.xlsx", header = 2)

In [283]:
final_matching = pd.read_csv('final_matching.csv').drop(['Unnamed: 0', 'Scores'], axis = 1)

In [284]:
spec_names = final_matching[final_matching['Delegates'] != final_matching['Loan Matches']]

In [285]:
#cleaning strings for constitutional convention delegates
delegates['state'] = [x.strip() for x in delegates['state']]
delegates['sign'] = [x.strip() for x in delegates['sign?']]
delegates['first name'] = [x.strip() for x in delegates['first name']]
delegates['last name'] = [x.strip() for x in delegates['last name']]

#might want to separate names with parentheses into last name 1/2
sumaltfirst = sum(["(" in x for x in delegates['first name']])
print("number of alt first names:", sumaltfirst)

sumaltlast = sum(["(" in x for x in delegates['last name']])
print("number of alt last names:", sumaltlast)

#handle special case
delegates.loc[40, 'last name 2'] = 'Fitzsimons (Fitzsimmons)'

#construct full names for delegate files
delegates['last name 2'] = [np.nan if "(" not in x else  x[x.find("(")+1:].replace(")","")  for x in delegates['last name']]
delegates['last name'] = delegates['last name'].apply(lambda x: re.sub(r'\([^)]*\)', '', x).strip())
delegates['full name 1'] = delegates['first name'] + " " + delegates['last name']
delegates['full name 2'] = delegates['first name'] + " " + delegates['last name 2']

number of alt first names: 0
number of alt last names: 4


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [286]:
#cleaning strings for state convention delegates
missing_ind = state_delegates[state_delegates['First Name'].apply(lambda x: type(x) != str)].index
state_delegates.loc[missing_ind, 'First Name'] = ['William', 'Ebenezer']

state_delegates['full name 1'] = state_delegates['First Name'] + " " + state_delegates['Last Name']

# Delegate Asset Database

In [287]:
delegates_assets = delegates[['full name 1', 'full name 2', 'sign']]
state_delegate_assets = state_delegates[['full name 1', 'State', 'Vote']]
state_delegate_assets.rename(columns = {"Vote":"sign", "State":"delegate_state"}, inplace = True)
total_assets = pd.concat([delegates_assets, state_delegate_assets]).reset_index(drop = True)

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  after removing the cwd from sys.path.


In [288]:
#amending existing cases

In [289]:
add_total_assets = total_assets[total_assets['full name 2'].apply(lambda x: not pd.isnull(x))]
new = add_total_assets.loc[[40]]
new['full name 2'] = 'Thomas Fitzsimmons'
add_total_assets.loc[40, 'full name 2'] = ' Thomas FitzSimons'
add_total_assets = pd.concat([add_total_assets, new])

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [290]:
total_assets['full name 2'] = total_assets['full name 1']
total_assets_full = pd.concat([total_assets, add_total_assets])
total_assets_full.sort_values('full name 1')

,delegate_state,full name 1,full name 2,sign
873,MA,A. Fisher,A. Fisher,yes
850,MA,A. Fisk,A. Fisk,no
790,MA,A. Fuller,A. Fuller,yes
816,MA,A. Parlin,A. Parlin,no
177,CT,Aaron Austin,Aaron Austin,yes
...,...,...,...,...
1557,NC,Zebedee Wood,Zebedee Wood,no
157,CT,"Zebulon Peck, Jun.","Zebulon Peck, Jun.",yes
1053,MA,Zenas Noble,Zenas Noble,no
579,NY,Zephaniah Platt,Zephaniah Platt,yes


In [291]:
merge_1 = pd.merge(total_assets_full.drop('full name 2', axis = 1), 
                   spec_names, left_on = 'full name 1', 
                   right_on = 'Delegates').drop('full name 1', axis = 1)
merge_2 = pd.merge(total_assets_full.drop('full name 1', axis = 1), 
                   spec_names, left_on = 'full name 2', 
                   right_on = 'Delegates').drop('full name 2', axis = 1)
merged = pd.concat([merge_1, merge_2]).drop_duplicates()

In [292]:
total_assets_full.columns = ['delegate_state', 'delegate name', 'match name', 'sign']
merged.columns =  ['delegate_state', 'sign',  'delegate name', 'match name']

In [293]:
total_assets_final = pd.concat([total_assets_full, merged])

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


# Loans Dataset Matching

In [294]:
#prepare loan dataset
loans = pd.read_csv("loan_office_certificates_9_states_cleaned.csv").drop('Unnamed: 0', axis = 1)

loans = loans.rename(columns = {"First Name 1 ":"First Name 1",
                                "Last Name 1 ":"Last Name 1",
                                "Specie Value ":"Specie Value"})

loans['full name 1'] = loans['First Name 1'] + " " + loans['Last Name 1']
loans['full name 2'] = loans['First Name 2'] + " " + loans['Last Name 2']
loans['full name 3'] = loans['First Name 3'] + " " + loans['Last Name 3']

In [295]:
#specie, face and num

In [296]:
loans['index'] = loans.index

loan_small_f1 = loans[['State', 'Year', 'Month', 'Day', 'full name 1', 'Face Value','Specie Value', 'notes', 'original text', 'index']]
loan_small_f2 = loans[['State', 'Year', 'Month', 'Day', 'full name 2', 'Face Value','Specie Value', 'notes', 'original text', 'index']]
loan_small_f3 = loans[['State', 'Year', 'Month', 'Day', 'full name 3', 'Face Value','Specie Value', 'notes', 'original text', 'index']]
loan_office_cert_assets = pd.merge(total_assets_final,
                                   loan_small_f1[loan_small_f1['full name 1'].apply(lambda x: not pd.isnull(x))],
                                   left_on = 'match name', right_on = 'full name 1').drop('full name 1', axis = 1)
add_to_assets = pd.merge(total_assets_final,
                         loan_small_f2[loan_small_f2['full name 2'].apply(lambda x: not pd.isnull(x))],
                         left_on = 'match name', right_on = 'full name 2').drop('full name 2', axis = 1)
loan_office_cert_assets = pd.concat([loan_office_cert_assets, add_to_assets])
add_to_assets = pd.merge(total_assets_final,
                         loan_small_f3[loan_small_f3['full name 3'].apply(lambda x: not pd.isnull(x))],
                         left_on = 'match name', right_on = 'full name 3').drop('full name 3', axis = 1)
loan_office_cert_assets = pd.concat([loan_office_cert_assets, add_to_assets])
loan_office_cert_assets = loan_office_cert_assets.drop_duplicates(['delegate name', 'index'])
loan_office_cert_assets['delegate_state'] = loan_office_cert_assets['delegate_state'].apply(lambda x: 'Constitution' if pd.isnull(x) else x)

loan_office_cert_assets_agg = loan_office_cert_assets.groupby(['delegate name',  'delegate_state', 'sign'])[['Face Value', 'Specie Value']].agg(['sum','count'])
loan_office_cert_assets_agg

Face Value       Specie Value      
                                            sum count          sum count
delegate name    delegate_state sign                                    
Aaron Austin     CT             yes        3700     3   180.066670     3
Aaron Wood       MA             no          600     1    66.725000     1
Abraham Faw      MD             yes         400     1    35.538890     1
Abraham Holmes   MA             no        13200     8   527.525000     8
Abraham Smith    MA             no          300     1   123.491700     1
...                                         ...   ...          ...   ...
William White    VA             no        11400    18  1197.584500    18
William Williams CT             yes       20600    46  5493.756885    46
William Wilson   PA             yes        2800     9   272.002770     9
William Woodhull NJ             yes        1000     2    51.069440     2
Williams John    RI             no       100300   163  9134.590290   163

[414 rows x 4 columns]

In [323]:
loan_office_cert_assets_agg.to_csv('Loan_Office_Certificates_Matched.csv')

# Pierce Certificates

In [297]:
#prepare pierce certificates
pierce = pd.read_excel("Data/Pierce_Certs_cleaned_2019.xlsx")

#clean first and last names
pierce = pierce[pierce['First'].apply(lambda x: not pd.isnull(x))]
pierce_ind = pierce[pierce['Last'].apply(lambda x: type(x) != str)].index
#fix issue with types for names
pierce.loc[pierce_ind, 'Last'] = ['True'] * len(pierce_ind)
#create full name
pierce['full name 1'] = pierce['First'] + " " + pierce['Last']

In [298]:
pierce_small_f1 = pierce[['State','full name 1', 'Value', 'CN']]
pierce_cert_assets = pd.merge(total_assets_final,
                              pierce_small_f1[pierce_small_f1['full name 1'].apply(lambda x: not pd.isnull(x))],
                              left_on = 'match name', right_on = 'full name 1').drop('full name 1', axis = 1)
pierce_cert_assets = pierce_cert_assets.drop_duplicates(['delegate name', 'CN'])
pierce_cert_assets['delegate_state'] = pierce_cert_assets['delegate_state'].apply(lambda x: 'Constitution' if pd.isnull(x) else x)

pierce_cert_assets_agg = pierce_cert_assets.groupby(['delegate name',  'delegate_state', 'sign'])[['Value']].agg(['sum','count'])
pierce_cert_assets_agg

Value      
                                            sum count
delegate name      delegate_state sign               
Aaron Hall         NH             yes     67.20     2
Aaron Wood         MA             no     120.60     2
Abraham Baldwin    Constitution   yes   3746.75     6
Abraham Smith      MA             no     290.20     5
Abraham Stout      PA             yes   2014.50     5
...                                         ...   ...
William White      VA             no    4272.54    26
William Williams   CT             yes   3765.59    45
William Wilson     PA             yes    334.70     7
Williams John      RI             no    6812.95    62
Zachariah Johnston VA             yes    147.20     1

[443 rows x 2 columns]

In [324]:
pierce_cert_assets_agg.to_csv('Pierce_Certificates_Matched.csv')

# Marine Certificates

In [299]:
#Read in file
marine = pd.read_excel("Data/Marine_Liquidated_Debt_Certificates.xlsx", header = 11)
cols = marine.columns = ["Page","JPEG number","Number","Letter","Date of the Certificate: Month",
                         "Date of the Certificate: Day","Date of the Certificate: Year","First name","Last name",
                         "Title","Time when the debt became due: Month","Time when the debt became due: Day",
                         "Time when the debt became due: Year","Dollars","90th","Total Dollars_1","Total Dollars_2",
                         "Line Strike Thorugh: Yes?","Line Strike Thorugh: Note","Notes"]
marine.columns = cols

marine = marine[marine['First name'].apply(lambda x: not pd.isnull(x))]

#create full name
marine['full name 1'] = marine['First name'] + " " + marine['Last name']

In [300]:
marine_small_f1 = marine[['full name 1', 'Number', 'Dollars', '90th']]
marine_cert_assets = pd.merge(total_assets_final,
                              marine_small_f1[marine_small_f1['full name 1'].apply(lambda x: not pd.isnull(x))],
                              left_on = 'match name', right_on = 'full name 1').drop('full name 1', axis = 1)
marine_cert_assets = marine_cert_assets.drop_duplicates(['delegate name', 'Number'])
marine_cert_assets['delegate_state'] = marine_cert_assets['delegate_state'].apply(lambda x: 'Constitution' if pd.isnull(x) else x)

marine_cert_assets_agg = marine_cert_assets.groupby(['delegate name',  'delegate_state', 'sign'])[['Dollars', '90th']].agg(['sum','count'])
marine_cert_assets_agg

Dollars         90th      
                                               sum count    sum count
delegate name          delegate_state sign                           
Brown John             RI             yes    262.0     1    0.0     0
David Bell             VA             no     108.0     1   14.0     1
David Rice             NC             yes     76.0     1   10.0     1
Davis John             RI             yes     35.0     1   25.0     1
James Neal             MA             no      42.0     1   24.0     1
John Brown             NC             no     262.0     1    0.0     0
John Davis             MA             yes     35.0     1   25.0     1
John Dunlap            MA             yes      1.0     1   85.0     1
John Hunter            SC             yes    310.0     1   51.0     1
John Jones             DE             yes     77.0     2   85.0     2
John K. Smith          MA             yes     23.0     1   85.0     1
John Rhea              NC             yes     43.0     1   57.0     1
John Smith             NY             yes     23.0     1   85.0     1
John Taylor            MA             no      72.0     1   45.0     1
John Williams          VA             yes     28.0     2  111.0     2
Jonathan Smith         MA             no     106.0     2  147.0     2
Samuel Johnston        NC             yes     52.0     1   45.0     1
Thomas Jones           MA             yes     95.0     1   62.0     1
William Hart           CT             yes     45.0     1   65.0     1
William Jones          MA             no    1871.0     5  132.0     2
William Morris         MD             yes    471.0     3    4.0     1
William Samuel Johnson Constitution   yes     52.0     1   45.0     1
William Scott          SC             yes     84.0     1   10.0     1
William Smith          SC             yes    636.0     1   72.0     1
Williams John          RI             no      28.0     2  111.0     2

In [325]:
marine_cert_assets_agg.to_csv('Marine_Certificates_Matched.csv')

# Connecticut

In [301]:
#Read in file
CT = pd.read_excel("Data/liquidated_debt_certificates_CT.xlsx", header = 11)
cols = CT.columns = ["Register Page","JPEG number","Number","Letter","Date of the Certificate: Month",
                     "Date of the Certificate: Day","Date of the Certificate: Year","First name","Last name",
                     "Title","First name 2","Last name 2","Title 2","Time when the debt became due: Month",
                     "Time when the debt became due: Day", "Time when the debt became due: Year","Dollars","90th",
                     "Line Strike Thorugh: Yes?","Line Strike Thorugh: Note","Notes"]
CT.columns = cols
CT = CT[CT['First name'].apply(lambda x: not pd.isnull(x))]

#create full name
CT['full name 1'] = CT['First name'] + " " + CT['Last name']
CT['full name 2'] = CT['First name 2'] + " " + CT['Last name 2']

In [302]:
CT_short = CT[['full name 1', 'full name 2', 'Number', 'Dollars', '90th']]
CT_short['Number'] = CT_short['Number'].apply(lambda x: 'CT_' + str(int(x)))

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


# Delaware

In [303]:
#Read in file
DE = pd.read_excel("Data/liquidated_debt_certificates_DE.xlsx", header = 11)
cols = DE.columns = ["JPEG number","Number","Letter","Date of the Certificate: Month",
                     "Date of the Certificate: Day","Date of the Certificate: Year","Title","First name","Last name",
                     "Time when the debt became due: Month",
                     "Time when the debt became due: Day", "Time when the debt became due: Year","Dollars","90th",
                     "Line Strike Through","Note"]
DE.columns = cols

DE = DE[DE['First name'].apply(lambda x: not pd.isnull(x))]

#create full name
DE['full name 1'] = DE['First name'] + " " + DE['Last name']

In [304]:
DE_short = DE[['full name 1', 'Number', 'Dollars', '90th']]
DE_short['Number'] = DE_short['Number'].apply(lambda x: 'DE_' + str(int(x)))
states_short = pd.concat([DE_short, CT_short])

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  This is separate from the ipykernel package so we can avoid doing imports until


# Massachusetts

In [305]:
#Read in file
MA = pd.read_excel("Data/liquidated_debt_certificates_MA.xlsx", header = 11)
cols = ["JPEG number","Page", "Number","Letter","Date of the Certificate: Month",
                     "Date of the Certificate: Day","Date of the Certificate: Year","First name","Last name","Title",
                     "First name 2","Last name 2", "Title 2","Time when the debt became due: Month",
                     "Time when the debt became due: Day", "Time when the debt became due: Year","Dollars","90th",
                     "Line Strike Through","Note"]
MA = MA[MA.columns[0:20]]
MA.columns = cols

MA = MA[MA['First name'].apply(lambda x: not pd.isnull(x))]

#create full name
MA['full name 1'] = MA['First name'] + " " + MA['Last name']
MA['full name 2'] = MA['First name 2'] + " " + MA['Last name 2']

In [306]:
MA_short = MA[['full name 1', 'full name 2', 'Number', 'Dollars', '90th']]
MA_short['Number'] = MA_short['Number'].apply(lambda x: 'MA_' + str(int(x)))
states_short = pd.concat([states_short, MA_short])

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  This is separate from the ipykernel package so we can avoid doing imports until


# New Hampshire

In [307]:
#Read in file
NH = pd.read_excel("Data/liquidated_debt_certificates_NH.xlsx", header = 11)
cols = ["Page","JPEG number","Number","Letter","Date of the Certificate: Month",
                         "Date of the Certificate: Day","Date of the Certificate: Year","First name","Last name",
                         "Title","Time when the debt became due: Month","Time when the debt became due: Day",
                         "Time when the debt became due: Year","Dollars","90th",
                         "Line Strike Thorugh: Yes?","Line Strike Thorugh: Note","Notes"]
NH.columns = cols

NH = NH[NH['First name'].apply(lambda x: not pd.isnull(x))]

#create full name
NH['full name 1'] = NH['First name'] + " " + NH['Last name']

In [308]:
NH_short = NH[['full name 1', 'Number', 'Dollars', '90th']]
NH_short['Number'] = NH_short['Number'].apply(lambda x: 'NH_' + str(int(x)))
states_short = pd.concat([states_short, NH_short])

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  This is separate from the ipykernel package so we can avoid doing imports until


# New Jersey

In [309]:
#Read in file
NJ = pd.read_excel("Data/liquidated_debt_certificates_NJ.xlsx", header = 10)
cols = ["JPEG number","Number","Letter","Date of the Certificate: Month",
        "Date of the Certificate: Day","Date of the Certificate: Year","Title","First name","Last name",
        "Title 2","First name 2","Last name 2","Time when the debt became due: Month",
        "Time when the debt became due: Day", "Time when the debt became due: Year","Dollars","90th",
        "Strike Through Number","Note"]
NJ = NJ[NJ.columns[0:19]]
NJ.columns = cols

NJ = NJ[NJ['First name'].apply(lambda x: not pd.isnull(x))]

#create full name
NJ['full name 1'] = NJ['First name'] + " " + NJ['Last name']
NJ['full name 2'] = NJ['First name 2'] + " " + NJ['Last name 2']

In [310]:
NJ_short = NJ[['full name 1', 'full name 2', 'Number', 'Dollars', '90th']]
NJ_short = NJ_short[NJ_short['Number'].apply(lambda x: not pd.isnull(x))]
NJ_short['Number'] = NJ_short['Number'].apply(lambda x: 'NJ_' + str(int(x)))
states_short = pd.concat([states_short, NJ_short])

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  after removing the cwd from sys.path.


# New York

In [311]:
#Read in file
NY = pd.read_excel("Data/liquidated_debt_certificates_NY.xlsx", header = 10)
cols = ["Page", "JPEG number","Number","Letter","Date of the Certificate: Month",
        "Date of the Certificate: Day","Date of the Certificate: Year","First name","Last name",
        "Title","First name 2","Last name 2","Title 2","Time when the debt became due: Month",
        "Time when the debt became due: Day", "Time when the debt became due: Year","Dollars","90th",
        "Strike Through Yes?","Note", "Notes"]
NY = NY[NY.columns[0:21]]
NY.columns = cols

NY = NY[NY['First name'].apply(lambda x: not pd.isnull(x))]

#create full name
NY['full name 1'] = NY['First name'] + " " + NY['Last name']
NY['full name 2'] = NY['First name 2'] + " " + NY['Last name 2']

In [312]:
NY_short = NY[['full name 1', 'full name 2', 'Number', 'Dollars', '90th']]
NY_short = NY_short[NY_short['Number'].apply(lambda x: not pd.isnull(x))]
NY_short['Number'] = NY_short['Number'].apply(lambda x: 'NY_' + str(int(x)))
states_short = pd.concat([states_short, NY_short])

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  after removing the cwd from sys.path.


# Pennsylvania P1

In [313]:
#Read in file
PA = pd.read_excel("Data/liquidated_debt_certificates_PA_story.xlsx", header = 11)
cols = ["JPEG number","Number","Letter","Date of the Certificate: Year",
        "Date of the Certificate: Month","Date of the Certificate: Date","Title","First name","Last name","Time when the debt became due: Year",
        "Time when the debt became due: Month","Time when the debt became due: Day",
        "Dollars","90th","10th", "Exchange","Amount in Specie_1", "Amount in Specie_2","Line Strike Thorugh?","Notes"]
PA = PA[PA.columns[1:21]]
PA.columns = cols

PA = PA[PA['First name'].apply(lambda x: not pd.isnull(x))]

#create full name
PA['full name 1'] = PA['First name'] + " " + PA['Last name']

In [314]:
PA_short_1 = PA[['full name 1', 'Number', 'Dollars', '90th']]
PA_short_1 = PA_short_1[PA_short_1['Number'].apply(lambda x: not pd.isnull(x))]
PA_short_1['Number'] = PA_short_1['Number'].apply(lambda x: 'PA_1_' + str(int(x)))
states_short = pd.concat([states_short, PA_short_1])

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  after removing the cwd from sys.path.


# Pennsylvania P2

In [315]:
#Read in file
PA2 = pd.read_excel("Data/liquidated_debt_certificates_PA_stelle.xlsx", header = 11)
cols = ["JPEG number","Number","Letter","Date of the Certificate: Year",
        "Date of the Certificate: Month","Date of the Certificate: Date","Title","First name","Last name","Title 2","First name 2","Last name 2",
        "Time when the debt became due: Year", "Time when the debt became due: Month","Time when the debt became due: Day",
        "Dollars","90th","Line Strike Thorugh?","Notes"]
PA2 = PA2[PA2.columns[1:21]]
PA2.columns = cols

PA2 = PA2[PA2['First name'].apply(lambda x: not pd.isnull(x))]

#create full name
PA2['full name 1'] = PA2['First name'] + " " + PA2['Last name']
PA2['full name 2'] = PA2['First name 2'] + " " + PA2['Last name 2']

In [316]:
PA_short_2 = PA2[['full name 1', 'full name 2', 'Number', 'Dollars', '90th']]
PA_short_2 = PA_short_2[PA_short_2['Number'].apply(lambda x: not pd.isnull(x))]
PA_short_2['Number'] = PA_short_2['Number'].apply(lambda x: 'PA_2_' + str(int(x)))
states_short = pd.concat([states_short, PA_short_2])

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  after removing the cwd from sys.path.


# Rhode Island

In [317]:
#Read in file
RI = pd.read_excel("Data/liquidated_debt_certificates_RI.xlsx", header = 12)
cols =  ["Page", "JPEG number","Number","Letter","Date of the Certificate: Month",
        "Date of the Certificate: Day","Date of the Certificate: Year","First name","Last name",
        "Title","First name 2","Last name 2","Title 2","Time when the debt became due: Month",
        "Time when the debt became due: Day", "Time when the debt became due: Year","Dollars","90th",
        "8th"]
RI = RI[RI.columns[0:19]]
RI.columns = cols

RI = RI[RI['First name'].apply(lambda x: not pd.isnull(x))]

#create full name
RI['full name 1'] = RI['First name'] + " " + RI['Last name']
RI['full name 2'] = RI['First name 2'] + " " + RI['Last name 2']

In [318]:
RI_short = RI[['full name 1', 'full name 2', 'Number', 'Dollars', '90th']]
RI_short = RI_short[RI_short['Number'].apply(lambda x: not pd.isnull(x))]
RI_short['Number'] = RI_short['Number'].apply(lambda x: 'RI_' + str(int(x)))
states_short = pd.concat([states_short, RI_short])

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  after removing the cwd from sys.path.


# Final Steps

In [322]:
state_cert_assets = pd.merge(total_assets_final,
                                   states_short[states_short['full name 1'].apply(lambda x: not pd.isnull(x))],
                                   left_on = 'match name', right_on = 'full name 1').drop('full name 1', axis = 1)
add_to_assets = pd.merge(total_assets_final,
                         states_short[states_short['full name 2'].apply(lambda x: not pd.isnull(x))],
                         left_on = 'match name', right_on = 'full name 2').drop('full name 2', axis = 1)
state_cert_assets = pd.concat([state_cert_assets, add_to_assets])
state_cert_assets = state_cert_assets.drop_duplicates(['delegate name', 'Number'])
state_cert_assets['delegate_state'] = state_cert_assets['delegate_state'].apply(lambda x: 'Constitution' if pd.isnull(x) else x)

state_cert_assets_agg = state_cert_assets.groupby(['delegate name',  'delegate_state', 'sign'])[['Dollars', '90th']].agg(['sum','count'])
state_cert_assets_agg

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  import sys


Dollars       90th      
                                           sum count  sum count
delegate name      delegate_state sign                         
Abraham Smith      MA             no       4.0     1    0     0
Abraham Stout      PA             yes     14.0     2  136     2
Albert Baker       NY             no      28.0     1    0     0
Alexander Hamilton Constitution   yes    160.0     1    0     0
Alexander Martin   Constitution   no     124.0     2  110     2
...                                        ...   ...  ...   ...
William White      VA             no     721.0    12  383    10
William Williams   CT             yes   1227.0    12  354     8
William Wilson     PA             yes      0.0     0    0     0
Williams John      RI             no    1400.0    14  436    11
Zephaniah Platt    NY             yes    284.0     1   26     1

[379 rows x 4 columns]

In [326]:
state_cert_assets_agg.to_csv('State_Certificates_Matched.csv')